In [ ]:
import re
import random

data_path = "model/human_text.txt"
data_path2 = "model/robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hola',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line.lower())) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line.lower())) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [2]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [3]:
print(pairs[:5])
print(input_docs[:5])

[('hola buenos días', 'buenos días en que puedo servirle'), ('tienes domicilio', 'claro que si que deseas pedir a domicilio'), ('que cuesta la arepa de huevo', 'la arepa tiene un costo de 2000 pesos sin incluir el domicilio'), ('y vendes chocolatico caliente', 'claro el chocolate un costo de 1000 sin leche'), ('y con leche', 'con lehe tiene un costo de 2000 pesos')]
['hola buenos días', 'tienes domicilio', 'que cuesta la arepa de huevo', 'y vendes chocolatico caliente', 'y con leche']


In [4]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 800
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='ADAM', loss='categorical_crossentropy', metrics=['accuracy'])
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('model/training_model.h5')

Epoch 1/800
21/21 [==============================] - 7s 134ms/step - loss: 1.5300 - accuracy: 0.0164 - val_loss: 1.2931 - val_accuracy: 0.0296
Epoch 2/800
21/21 [==============================] - ETA: 0s - loss: 1.2676 - accuracy: 0.03 - 1s 50ms/step - loss: 1.2685 - accuracy: 0.0330 - val_loss: 1.2793 - val_accuracy: 0.0296
Epoch 3/800
21/21 [==============================] - 1s 49ms/step - loss: 1.2158 - accuracy: 0.0347 - val_loss: 1.2747 - val_accuracy: 0.0286
Epoch 4/800
21/21 [==============================] - 1s 49ms/step - loss: 1.3060 - accuracy: 0.0321 - val_loss: 1.2767 - val_accuracy: 0.0291
Epoch 5/800
21/21 [==============================] - 1s 49ms/step - loss: 1.2254 - accuracy: 0.0314 - val_loss: 1.2676 - val_accuracy: 0.0291
Epoch 6/800
21/21 [==============================] - 1s 49ms/step - loss: 1.2305 - accuracy: 0.0302 - val_loss: 1.2666 - val_accuracy: 0.0291
Epoch 7/800
21/21 [==============================] - 1s 49ms/step - loss: 1.2171 - accuracy: 0.0328 - val

21/21 [==============================] - 1s 49ms/step - loss: 0.9566 - accuracy: 0.0803 - val_loss: 1.1659 - val_accuracy: 0.0576
Epoch 59/800
21/21 [==============================] - 1s 48ms/step - loss: 0.9750 - accuracy: 0.0857 - val_loss: 1.1594 - val_accuracy: 0.0586
Epoch 60/800
21/21 [==============================] - 1s 49ms/step - loss: 0.9199 - accuracy: 0.0816 - val_loss: 1.1582 - val_accuracy: 0.0581
Epoch 61/800
21/21 [==============================] - 1s 49ms/step - loss: 0.9536 - accuracy: 0.0806 - val_loss: 1.1571 - val_accuracy: 0.0586
Epoch 62/800
21/21 [==============================] - 1s 48ms/step - loss: 0.9101 - accuracy: 0.0829 - val_loss: 1.1491 - val_accuracy: 0.0607
Epoch 63/800
21/21 [==============================] - 1s 49ms/step - loss: 0.8786 - accuracy: 0.0827 - val_loss: 1.1484 - val_accuracy: 0.0632
Epoch 64/800
21/21 [==============================] - 1s 49ms/step - loss: 0.9073 - accuracy: 0.0878 - val_loss: 1.1411 - val_accuracy: 0.0643
Epoch 65/800

21/21 [==============================] - 1s 49ms/step - loss: 0.6028 - accuracy: 0.1353 - val_loss: 1.0906 - val_accuracy: 0.0806
Epoch 116/800
21/21 [==============================] - 1s 49ms/step - loss: 0.6198 - accuracy: 0.1400 - val_loss: 1.0832 - val_accuracy: 0.0796
Epoch 117/800
21/21 [==============================] - 1s 50ms/step - loss: 0.6315 - accuracy: 0.1386 - val_loss: 1.0874 - val_accuracy: 0.0806
Epoch 118/800
21/21 [==============================] - 1s 49ms/step - loss: 0.6373 - accuracy: 0.1449 - val_loss: 1.0827 - val_accuracy: 0.0801
Epoch 119/800
21/21 [==============================] - 1s 50ms/step - loss: 0.6234 - accuracy: 0.1407 - val_loss: 1.0866 - val_accuracy: 0.0806
Epoch 120/800
21/21 [==============================] - 1s 49ms/step - loss: 0.6236 - accuracy: 0.1430 - val_loss: 1.0784 - val_accuracy: 0.0826
Epoch 121/800
21/21 [==============================] - 1s 49ms/step - loss: 0.5964 - accuracy: 0.1451 - val_loss: 1.0819 - val_accuracy: 0.0811
Epoch 

Epoch 172/800
21/21 [==============================] - 1s 49ms/step - loss: 0.4702 - accuracy: 0.1832 - val_loss: 1.0869 - val_accuracy: 0.0862
Epoch 173/800
21/21 [==============================] - 1s 49ms/step - loss: 0.4796 - accuracy: 0.1852 - val_loss: 1.0853 - val_accuracy: 0.0857
Epoch 174/800
21/21 [==============================] - 1s 49ms/step - loss: 0.4723 - accuracy: 0.1901 - val_loss: 1.0865 - val_accuracy: 0.0862
Epoch 175/800
21/21 [==============================] - 1s 49ms/step - loss: 0.5009 - accuracy: 0.1876 - val_loss: 1.0907 - val_accuracy: 0.0857
Epoch 176/800
21/21 [==============================] - 1s 48ms/step - loss: 0.4798 - accuracy: 0.1816 - val_loss: 1.0872 - val_accuracy: 0.0852
Epoch 177/800
21/21 [==============================] - 1s 49ms/step - loss: 0.4584 - accuracy: 0.1925 - val_loss: 1.0765 - val_accuracy: 0.0857
Epoch 178/800
21/21 [==============================] - 1s 49ms/step - loss: 0.4591 - accuracy: 0.1895 - val_loss: 1.0817 - val_accuracy:

Epoch 229/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3368 - accuracy: 0.2116 - val_loss: 1.0943 - val_accuracy: 0.0857
Epoch 230/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3594 - accuracy: 0.2137 - val_loss: 1.0990 - val_accuracy: 0.0872
Epoch 231/800
21/21 [==============================] - 1s 48ms/step - loss: 0.3386 - accuracy: 0.2177 - val_loss: 1.0909 - val_accuracy: 0.0887
Epoch 232/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3435 - accuracy: 0.2152 - val_loss: 1.0928 - val_accuracy: 0.0887
Epoch 233/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3215 - accuracy: 0.2065 - val_loss: 1.0985 - val_accuracy: 0.0847
Epoch 234/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3348 - accuracy: 0.2142 - val_loss: 1.1008 - val_accuracy: 0.0872
Epoch 235/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3182 - accuracy: 0.2102 - val_loss: 1.1025 - val_accuracy:

...

Epoch 570/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1418 - accuracy: 0.2500 - val_loss: 1.1106 - val_accuracy: 0.0918
Epoch 571/800
21/21 [==============================] - 1s 50ms/step - loss: 0.1363 - accuracy: 0.2453 - val_loss: 1.1113 - val_accuracy: 0.0913
Epoch 572/800
21/21 [==============================] - 1s 51ms/step - loss: 0.1399 - accuracy: 0.2545 - val_loss: 1.1116 - val_accuracy: 0.0908
Epoch 573/800
21/21 [==============================] - 1s 57ms/step - loss: 0.1274 - accuracy: 0.2347 - val_loss: 1.1097 - val_accuracy: 0.0913
Epoch 574/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1150 - accuracy: 0.2325 - val_loss: 1.1115 - val_accuracy: 0.0913
Epoch 575/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1428 - accuracy: 0.2567 - val_loss: 1.1104 - val_accuracy: 0.0918
Epoch 576/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1252 - accuracy: 0.2468 - val_loss: 1.1118 - val_accuracy:

Epoch 627/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1602 - accuracy: 0.2364 - val_loss: 1.1042 - val_accuracy: 0.0923
Epoch 628/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1703 - accuracy: 0.2456 - val_loss: 1.1198 - val_accuracy: 0.0918
Epoch 629/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1447 - accuracy: 0.2300 - val_loss: 1.1912 - val_accuracy: 0.0729
Epoch 630/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3032 - accuracy: 0.2309 - val_loss: 1.1005 - val_accuracy: 0.0857
Epoch 631/800
21/21 [==============================] - 1s 49ms/step - loss: 0.2585 - accuracy: 0.2204 - val_loss: 1.0958 - val_accuracy: 0.0816
Epoch 632/800
21/21 [==============================] - 1s 49ms/step - loss: 0.2700 - accuracy: 0.2281 - val_loss: 1.1618 - val_accuracy: 0.0801
Epoch 633/800
21/21 [==============================] - 1s 49ms/step - loss: 0.3569 - accuracy: 0.2156 - val_loss: 1.1525 - val_accuracy:

Epoch 684/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1296 - accuracy: 0.2352 - val_loss: 1.0045 - val_accuracy: 0.1030
Epoch 685/800
21/21 [==============================] - 1s 50ms/step - loss: 0.1560 - accuracy: 0.2567 - val_loss: 1.0065 - val_accuracy: 0.1020
Epoch 686/800
21/21 [==============================] - 1s 50ms/step - loss: 0.1303 - accuracy: 0.2391 - val_loss: 1.0038 - val_accuracy: 0.1025
Epoch 687/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1217 - accuracy: 0.2490 - val_loss: 1.0090 - val_accuracy: 0.1015
Epoch 688/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1274 - accuracy: 0.2363 - val_loss: 1.0014 - val_accuracy: 0.1020
Epoch 689/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1521 - accuracy: 0.2581 - val_loss: 1.0068 - val_accuracy: 0.1025
Epoch 690/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1351 - accuracy: 0.2283 - val_loss: 1.0045 - val_accuracy:

Epoch 741/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1218 - accuracy: 0.2542 - val_loss: 1.0526 - val_accuracy: 0.0969
Epoch 742/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1289 - accuracy: 0.2695 - val_loss: 1.0536 - val_accuracy: 0.0969
Epoch 743/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1209 - accuracy: 0.2593 - val_loss: 1.0520 - val_accuracy: 0.0964
Epoch 744/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1021 - accuracy: 0.2411 - val_loss: 1.0561 - val_accuracy: 0.0964
Epoch 745/800
21/21 [==============================] - 1s 49ms/step - loss: 0.0939 - accuracy: 0.2402 - val_loss: 1.0517 - val_accuracy: 0.0964
Epoch 746/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1086 - accuracy: 0.2422 - val_loss: 1.0533 - val_accuracy: 0.0984
Epoch 747/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1004 - accuracy: 0.2454 - val_loss: 1.0550 - val_accuracy:

Epoch 798/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1076 - accuracy: 0.2585 - val_loss: 1.0772 - val_accuracy: 0.0943
Epoch 799/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1038 - accuracy: 0.2567 - val_loss: 1.0780 - val_accuracy: 0.0948
Epoch 800/800
21/21 [==============================] - 1s 49ms/step - loss: 0.1033 - accuracy: 0.2544 - val_loss: 1.0738 - val_accuracy: 0.0943


In [ ]:
training_model.save('model/training_model2.h5')

In [5]:
from keras.models import load_model
training_model = load_model('model/training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
          #Predicting output tokens with probabilities and states
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
          if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
    #Update the target sequence
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [6]:
class ChatBot:
  negative_responses = ("no", "negativo", "nada", "nop", "nopi")
  exit_commands = ("chao", "adios", "nos vemos",
                 "suerte", "chaito", "bye", "chaolin")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hola, bienvenido en que te puedo ayudar?\n")
    
    if user_response in self.negative_responses:
      print("Ok, perfecto, que tengas un hermoso día!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Que tengas un hermoso día!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Buenos días
 el almuerzo es chuzo de pollo 
tienes domicilio?
 claro que si que deseas pedir a domicilio
que cuesta la arepa de huevo?
 la arepa tiene un costo de 2000 pesos
y vendes chocolatico caliente?
 claro el chocolate un costo de 1000 sin
Y con leche?
 con lehe tiene un costo de 2000 pesos
Gaseosa manejas?
 si robalo y trucha 
listo, entonces me envias la arepa de huevo y una coca cola bien fria.
 esta bién me envias tu dirección barrio
Buenas noches
 asi es cuántos huevitos desea 
Deseo una pizza por favor
 claro que si indicame el tamaño y el sabor
Se puede mitad y mitad?
 lo siento soy un robot todo depende de
y que precio tiene mitad hawayana y mitad de peperoni?
 lo siento soy un robot el precio aproximado
Ok, me envias una pizza asi por favor
 el domicilio tiene un costo de 5000 
una cocacola litro y cuarto
 esta bién me envias tu dirección barrio
chica tienes servicio
 claro que si nuestro horario de atención
El chuzo de pollo que 

In [7]:
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Buenos días
 el almuerzo es chuzo de pollo 
Hola, Buenos días
 el menú del día cuesta 10000 
tienes domicilio?
 claro que si que deseas pedir a domicilio
Deseo una pizza por favor
 claro que si indicame el tamaño y el sabor
hola
 hola en que puedo servirte 
Que tengas un hermoso día!
 cuantas total a 
Hola
 cuantas arepas desea 
cuatro por favor
 claro vendemos si necesitas que puedo
chao
Que tengas un hermoso día!
